In [7]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
import datetime


In [8]:
df = pd.read_csv('./train.csv/train.csv')
df.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0
1,1,02/14/2008,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,3.0,0,0,0,0,0,0,0,0,6923131.0
2,2,03/09/2013,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,3.0,0,0,0,0,0,0,0,0,2055379.0
3,3,02/02/2012,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,7.5,25,12,10,6,18,12,12,6,2675511.0
4,4,05/09/2009,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3.0,5,1,3,2,3,4,3,3,4316715.0


In [9]:
X = pd.get_dummies(df, columns = ['City','City Group', 'Type'])

In [10]:
X['OpendFor_days'] = X['Open Date'].apply(lambda x: 
                                          int(abs(datetime.datetime.strptime(x, '%m/%d/%Y') - 
                                                            datetime.datetime.today()).days))
X.drop(['Open Date', 'Id'], axis = 1, inplace = True)
X.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,City_Uşak,City_İstanbul,City_İzmir,City_Şanlıurfa,City Group_Big Cities,City Group_Other,Type_DT,Type_FC,Type_IL,OpendFor_days
0,4,5.0,4.0,4.0,2,2,5,4,5,5,...,0,1,0,0,1,0,0,0,1,8014
1,4,5.0,4.0,4.0,1,2,5,5,5,5,...,0,0,0,0,1,0,0,1,0,4880
2,2,4.0,2.0,5.0,2,3,5,5,5,5,...,0,0,0,0,0,1,0,0,1,3030
3,6,4.5,6.0,6.0,4,4,10,8,10,10,...,0,0,0,0,0,1,0,0,1,3431
4,3,4.0,3.0,4.0,2,2,5,5,5,5,...,0,0,0,0,0,1,0,0,1,4430


In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 78 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   P1                     137 non-null    int64  
 1   P2                     137 non-null    float64
 2   P3                     137 non-null    float64
 3   P4                     137 non-null    float64
 4   P5                     137 non-null    int64  
 5   P6                     137 non-null    int64  
 6   P7                     137 non-null    int64  
 7   P8                     137 non-null    int64  
 8   P9                     137 non-null    int64  
 9   P10                    137 non-null    int64  
 10  P11                    137 non-null    int64  
 11  P12                    137 non-null    int64  
 12  P13                    137 non-null    float64
 13  P14                    137 non-null    int64  
 14  P15                    137 non-null    int64  
 15  P16   

In [12]:
y = X['revenue']
X.drop(['revenue'], axis = 1)

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,City_Uşak,City_İstanbul,City_İzmir,City_Şanlıurfa,City Group_Big Cities,City Group_Other,Type_DT,Type_FC,Type_IL,OpendFor_days
0,4,5.0,4.0,4.0,2,2,5,4,5,5,...,0,1,0,0,1,0,0,0,1,8014
1,4,5.0,4.0,4.0,1,2,5,5,5,5,...,0,0,0,0,1,0,0,1,0,4880
2,2,4.0,2.0,5.0,2,3,5,5,5,5,...,0,0,0,0,0,1,0,0,1,3030
3,6,4.5,6.0,6.0,4,4,10,8,10,10,...,0,0,0,0,0,1,0,0,1,3431
4,3,4.0,3.0,4.0,2,2,5,5,5,5,...,0,0,0,0,0,1,0,0,1,4430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,2,3.0,3.0,5.0,4,2,4,4,4,4,...,0,0,0,0,0,1,0,1,0,4748
133,4,5.0,4.0,4.0,2,3,5,4,4,5,...,0,0,1,0,1,0,0,1,0,5370
134,3,4.0,4.0,4.0,2,3,5,5,5,5,...,0,0,0,0,0,1,0,1,0,5466
135,4,5.0,4.0,5.0,2,2,5,5,5,5,...,0,1,0,0,1,0,0,1,0,3892


In [13]:
XGB_Model = xgb.XGBRegressor()
XGB_Model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
# to be removed
def rootMSE(true, predicted ):
    sum = 0
    for i in range(len(true)):
        sum = sum + (true[i] - predicted[i])**2
    return (sum/len(true))**0.5

rootMSE(list(y_test), XGB_Model.predict(X_test))

NameError: name 'y_test' is not defined

In [15]:
params = {
    'max_depth': [3,4,5],
    'learning_rate': [0.0625,0.125, 0.25, 0.33, 0.5, 0.625, 0.875, 1],
    'gamma': [0.1, 0.25, 0.5, 1, 2 ],
    
}
grid = GridSearchCV(estimator = XGB_Model, param_grid = params)
grid.fit(X, y)

GridSearchCV(estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0, gpu_id=-1,
                                    importance_type='gain',
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=6, min_child_weight=1,
                                    missing=nan, monotone_constraints='()',
                                    n_estimators=100, n_jobs=4,
                                    num_parallel_tree=1, random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, subsample=1,
                                    tree_method='exact', validate_parameters=1,
                                    verbosity=None),
             param_grid={

In [16]:
grid.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.33, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
XGB_Model_optimized = xgb.XGBRegressor(gamma = 0.1, max_depth = 5, learning_rate=0.33)
XGB_Model_optimized.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.33, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

For Test data

In [19]:
test_df = pd.read_csv('./test.csv/test.csv')
test_df.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,...,2.0,3.0,0,0,0,0,0,0,0,0
1,1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,...,1.0,3.0,0,0,0,0,0,0,0,0
2,2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,...,2.0,3.0,0,0,0,0,0,0,0,0
3,3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,...,2.0,3.0,0,4,0,0,0,0,0,0
4,4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,...,5.0,3.0,0,0,0,0,0,0,0,0


In [20]:
test_X = pd.get_dummies(test_df, columns = ['City','City Group', 'Type'])

In [21]:
test_X['OpendFor_days'] = test_X['Open Date'].apply(lambda x: 
                                          int(abs(datetime.datetime.strptime(x, '%m/%d/%Y') - 
                                                            datetime.datetime.today()).days))
test_X.drop(['Open Date', 'Id'], axis = 1, inplace = True)
test_X.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,City_İstanbul,City_İzmir,City_Şanlıurfa,City Group_Big Cities,City Group_Other,Type_DT,Type_FC,Type_IL,Type_MB,OpendFor_days
0,1,4.0,4.0,4.0,1,2,5,4,5,5,...,0,0,0,0,1,0,1,0,0,3807
1,3,4.0,4.0,4.0,2,2,5,3,4,4,...,0,0,0,0,1,0,0,1,0,3752
2,3,4.0,4.0,4.0,2,2,5,4,4,5,...,0,0,0,1,0,0,1,0,0,2795
3,2,4.0,4.0,4.0,2,3,5,4,5,4,...,0,0,0,0,1,0,0,1,0,2972
4,2,4.0,4.0,4.0,1,2,5,4,5,4,...,0,0,0,0,1,0,1,0,0,2886


In [22]:
test_X = X[X.columns]

In [23]:
XGB_Model_optimized.predict(test_X)

array([ 5653664. ,  6922910. ,  2055486.6,  2675326.8,  4316571.5,
        5017377.5,  5166365. ,  4492062.5,  4952348.5,  5443843.5,
        3744935.5,  5161335. ,  1734782.2,  4808030. ,  1999440.2,
        3218875.2, 19696538. ,  8213503. ,  5337484. ,  2022220.8,
        5525661. ,  1150142.5,  3956007.5,  2998680.5,  8903935. ,
        3778122. ,  2267429. ,  5435258. ,  4706446.5,  3447780.2,
        3008184.8,  2058717.8,  3261860.8,  2371123.8,  4100106.8,
        3903834.8,  3027791.2,  2740947.8,  5461356.5,  3818135.8,
        6836349.5,  4554128. ,  4952907.5,  2733139.2,  4263866.5,
        3753306. ,  4250632. ,  7865531.5,  7201663. ,  8630694. ,
        2156312.5,  3273423.5,  1763309.4,  5595763.5,  4651795.5,
        6941145.5,  2390185.5,  3258874. ,  3004324.2,  3601399.5,
        3165086. ,  4350435. ,  5286526. ,  1619539.8,  2551142.8,
        1756144.4,  1881957.2,  4052841.8,  3836761.2,  3452282. ,
        3982039. ,  4136746. ,  2525352. ,  3348029.2,  550078

In [24]:
test_X.shape

(137, 78)

In [25]:
X.shape

(137, 78)

In [28]:
a = list(set(X.columns).intersection(set(test_X.columns)))